In [2]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

print(result)

42


In [3]:
# (p. 236) Linear regression with TensorFlow on the housing data set
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X)
# standard normal equation using TF API
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    
print(theta_value)

[[ -3.74651413e+01]
 [  4.35734153e-01]
 [  9.33829229e-03]
 [ -1.06622010e-01]
 [  6.44106984e-01]
 [ -4.25131839e-06]
 [ -3.77322501e-03]
 [ -4.26648885e-01]
 [ -4.40514028e-01]]


In [6]:
# (p. 237) Gradient descent with TF
n_epochs = 2000
learning_rate = 0.01

from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
scaled_housing_data_plus_bias = std_scaler.fit_transform(housing_data_plus_bias)

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, n], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)
# this is one learning step:
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)    
    best_theta = theta.eval()
    print("Best theta:", best_theta)

Epoch 0 MSE = 8.06203
Epoch 100 MSE = 4.96944
Epoch 200 MSE = 4.88703
Epoch 300 MSE = 4.86446
Epoch 400 MSE = 4.84949
Epoch 500 MSE = 4.83842
Epoch 600 MSE = 4.83012
Epoch 700 MSE = 4.82391
Epoch 800 MSE = 4.81922
Epoch 900 MSE = 4.81565
Epoch 1000 MSE = 4.81293
Epoch 1100 MSE = 4.81084
Epoch 1200 MSE = 4.80924
Epoch 1300 MSE = 4.80799
Epoch 1400 MSE = 4.80702
Epoch 1500 MSE = 4.80627
Epoch 1600 MSE = 4.80567
Epoch 1700 MSE = 4.80522
Epoch 1800 MSE = 4.80482
Epoch 1900 MSE = 4.80454
Best theta: [[ -4.89522219e-01   2.10625648e-01  -9.79861259e-01   2.98736095e-01
   -6.26909733e-01   8.27973604e-01   9.60164070e-01  -6.50327206e-02]
 [  8.45440805e-01   8.41708839e-01   8.55238080e-01   8.62918615e-01
    8.66773069e-01   8.48532498e-01   8.56408894e-01   7.82870054e-01]
 [  1.24781199e-01   1.23740420e-01   1.28128588e-01   1.28970012e-01
    1.34137064e-01   1.24078609e-01   1.29976422e-01   1.15455113e-01]
 [ -2.89689958e-01  -2.83276558e-01  -3.05375189e-01  -3.21005881e-01
   -3.1

In [7]:
# (p. 238) same, but use TF for determining the gradients to make more efficient
n_epochs = 2000
learning_rate = 0.01

from sklearn.preprocessing import StandardScaler

std_scaler = StandardScaler()
scaled_housing_data_plus_bias = std_scaler.fit_transform(housing_data_plus_bias)

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, n], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")


# Here's the difference to the previous cell:
gradients = tf.gradients(mse, [theta])[0]

# this is one learning step:
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)    
    best_theta = theta.eval()
    print("Best theta:", best_theta)

Epoch 0 MSE = 7.58738
Epoch 100 MSE = 6.33017
Epoch 200 MSE = 5.70856
Epoch 300 MSE = 5.38441
Epoch 400 MSE = 5.20672
Epoch 500 MSE = 5.10477
Epoch 600 MSE = 5.0437
Epoch 700 MSE = 5.00549
Epoch 800 MSE = 4.98047
Epoch 900 MSE = 4.96326
Epoch 1000 MSE = 4.95077
Epoch 1100 MSE = 4.94121
Epoch 1200 MSE = 4.93352
Epoch 1300 MSE = 4.92703
Epoch 1400 MSE = 4.92137
Epoch 1500 MSE = 4.91629
Epoch 1600 MSE = 4.91162
Epoch 1700 MSE = 4.90728
Epoch 1800 MSE = 4.90318
Epoch 1900 MSE = 4.89934
Best theta: [[-0.11712837  0.53153443 -0.14883876 -0.18799949 -0.14060116 -0.88531971
  -0.13820291 -0.78350902]
 [ 0.88760036  0.55963671  0.7532382   0.5534566   0.80381     0.7586754
   0.85988253  0.95347047]
 [ 0.1985449   0.22024105  0.27843335  0.1802434   0.22131379  0.18268508
   0.13371889  0.20345362]
 [-0.23914604  0.53951705  0.16831592  0.4659254  -0.01822526  0.03662416
  -0.30070025 -0.39503318]
 [ 0.21951514 -0.50196099 -0.17858735 -0.3912054   0.01668435 -0.02695157
   0.31851643  0.3640991